In [ ]:
import sagemaker
import os

In [ ]:
import pathlib as pl

In [ ]:
import datetime

In [ ]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import TensorBoardOutputConfig

In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
upload_data = False
if upload_data:
    data_input_path = sess.upload_data('data/severstal-binary')
else:
    data_input_path = "s3://sagemaker-us-east-2-475496805360/data"
print(data_input_path)

In [ ]:
training_input_path = data_input_path + "/train"
validation_input_path = data_input_path + "/validate"

In [ ]:
time_now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_logs_dir = f"s3://{sess.default_bucket()}/tensorboard_logs_v2/fit/{time_now}"

In [ ]:
tensorboard_logs_dir

In [ ]:
tensorboard_output_config = TensorBoardOutputConfig(s3_output_path=tensorboard_logs_dir)

In [ ]:
tf_estimator = TensorFlow(
    entry_point='fine_tune_eff_net.py', 
    role=role,
    instance_count=1, 
    instance_type='local',  # local 'ml.g4dn.xlarge'
    framework_version='2.3', 
    py_version='py37',
    script_mode=True,
    hyperparameters={
        'epochs': 1,
        "steps-per-epoch": 1,
        "log-dir": tensorboard_logs_dir
    },
    tensorboard_output_config=tensorboard_output_config,
)

In [ ]:
tf_estimator.fit(
    {
        'training': training_input_path, 
        'validation': validation_input_path, 
    }
)

In [ ]:
file_list = [
    pl.Path(f)
    for f in sess.list_s3_files(sess.default_bucket(), "tensorboard_logs_v2/fit/")
    if not f.endswith("/")
]

In [ ]:
for f in file_list:
    print(f)
    sess.download_data(str(f.parent), sess.default_bucket(), str(f))

In [ ]:
!tensorboard --logdir sagemaker-image-classification/tensorboard_logs_v2/fit

In [ ]:
cd tensorboard_logs_v2

In [ ]:
cd fit

In [ ]:
!tensorboard --logdir .

In [ ]:
!tensorboard --help